In [1]:
import os
from smosaic import mosaic

In [2]:
path_dir = os.path.abspath("")

In [3]:
result = mosaic(
    name="sao-paulo",
    data_dir=os.path.join(path_dir),
    collection="AMZ1-WFI-L4-SR-1", 
    grid="br_states",
    grid_id="SP",
    output_dir=os.path.join(path_dir, "output"),   
    mosaic_method="lcf", 
    start_year=2025,
    start_month=1,
    start_day=1,
    duration_months=5, 
    bands=["BAND4"]
)

Downloading... : 100%|██████████| 105/105 [00:26<00:00,  3.91 itens/s]


Successfully download 105 files to AMZ1-WFI-L4-SR-1
Building BAND4 mosaic using 6 scenes from the AMZ1-WFI-L4-SR-1
Building CMASK mosaic using 6 scenes from the AMZ1-WFI-L4-SR-1
Successfully merged 4 files
Successfully merged 4 files
Successfully merged 4 files
Successfully merged 4 files
Successfully merged 4 files
Successfully merged 4 files
Successfully merged 6 files
Clipped raster saved to: /Volumes/Extreme SSD/smosaic/output/mosaic-amz1-band4-sao-paulo-5m.tif
